In [1]:
import os
import numpy as np
from torchvision import datasets, models, transforms
import torch
from torch.utils.data import DataLoader, Dataset
from torchvision.transforms import InterpolationMode
from PIL import Image
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import random

In [2]:
# Set fixed seed for reproducibility
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [3]:
set_seed(42)

In [4]:
# Define dataset paths and parameters
dataset_dir = "/kaggle/input/aug-v100"  # Replace with your dataset path
train_folder = os.path.join(dataset_dir, "output")
val_folder = os.path.join(dataset_dir, "val")
test_folder = os.path.join(dataset_dir, "test")
folders = ["Diabetic Retinopathy", "Glaucoma", "Healthy", "Macular Scar", "Myopia"]
image_size = 224  # Input image size for ResNet50

In [5]:
# Function to calculate dataset mean and std
def calculate_mean_std(dataset):
    loader = DataLoader(dataset, batch_size=64, shuffle=False)
    mean = torch.zeros(3)
    std = torch.zeros(3)
    for images, _ in loader:
        images = images.view(images.size(0), images.size(1), -1)
        mean += images.mean(2).sum(0)
        std += images.std(2).sum(0)
    mean /= len(dataset)
    std /= len(dataset)
    return mean, std

In [6]:
# Custom dataset class
class CustomDataset(Dataset):
    def __init__(self, folder_path, transform=None):
        self.folder_path = folder_path
        self.transform = transform
        self.data = []
        for class_label, class_name in enumerate(folders):
            class_folder = os.path.join(folder_path, class_name)
            if os.path.exists(class_folder):
                for img_name in os.listdir(class_folder):
                    img_path = os.path.join(class_folder, img_name)
                    self.data.append((img_path, class_label))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path, label = self.data[idx]
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, label

In [7]:
# Create datasets for mean/std calculation
combined_dataset = CustomDataset(train_folder, transform=transforms.ToTensor())
mean, std = calculate_mean_std(combined_dataset)
print(f"Dataset Mean: {mean}, Std: {std}")

Dataset Mean: tensor([0.5828, 0.4277, 0.2689]), Std: tensor([0.3004, 0.2478, 0.1858])


In [8]:
# Update transforms with calculated mean and std
transform = transforms.Compose([
    transforms.Resize((image_size, image_size), interpolation=InterpolationMode.BILINEAR),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean.tolist(), std=std.tolist())
])

In [9]:
# Create datasets for train, validation, and test
train_dataset = CustomDataset(train_folder, transform=transform)
val_dataset = CustomDataset(val_folder, transform=transform)
test_dataset = CustomDataset(test_folder, transform=transform)

In [10]:
# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)


In [11]:
# Load pretrained VGG16 model
model = models.vgg16(pretrained=True)
num_features = model.classifier[-1].in_features
model.classifier[-1] = nn.Sequential(
    nn.Dropout(0.5),  # Dropout with 50% probability
    nn.Linear(num_features, len(folders))
)



/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:02<00:00, 192MB/s]


In [12]:
# Training setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)  # L2 regularization with weight_decay

# Training loop
num_epochs = 50
best_val_accuracy = 0.0

In [13]:
for epoch in range(num_epochs):
    # Training phase
    model.train()
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    train_accuracy = 100 * correct / total

    # Validation phase
    model.eval()
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()

    val_accuracy = 100 * val_correct / val_total

    # Print epoch results
    print(f"Epoch {epoch+1}/{num_epochs}, "
          f"Train Accuracy: {train_accuracy:.2f}%, "
          f"Validation Accuracy: {val_accuracy:.2f}%")

    # Save the best model
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        torch.save(model.state_dict(), "best_resnet50_model.pth")

Epoch 1/50, Train Accuracy: 25.73%, Validation Accuracy: 31.20%
Epoch 2/50, Train Accuracy: 33.56%, Validation Accuracy: 48.14%
Epoch 3/50, Train Accuracy: 43.01%, Validation Accuracy: 50.62%
Epoch 4/50, Train Accuracy: 47.09%, Validation Accuracy: 48.76%
Epoch 5/50, Train Accuracy: 50.83%, Validation Accuracy: 50.62%
Epoch 6/50, Train Accuracy: 52.89%, Validation Accuracy: 48.97%
Epoch 7/50, Train Accuracy: 55.49%, Validation Accuracy: 45.87%
Epoch 8/50, Train Accuracy: 57.57%, Validation Accuracy: 54.34%
Epoch 9/50, Train Accuracy: 60.61%, Validation Accuracy: 59.71%
Epoch 10/50, Train Accuracy: 62.02%, Validation Accuracy: 55.99%
Epoch 11/50, Train Accuracy: 64.41%, Validation Accuracy: 63.84%
Epoch 12/50, Train Accuracy: 65.37%, Validation Accuracy: 62.19%
Epoch 13/50, Train Accuracy: 67.36%, Validation Accuracy: 62.40%
Epoch 14/50, Train Accuracy: 68.66%, Validation Accuracy: 64.46%
Epoch 15/50, Train Accuracy: 70.01%, Validation Accuracy: 63.64%
Epoch 16/50, Train Accuracy: 70.73

In [14]:
# Load the best model for testing
model.load_state_dict(torch.load("best_resnet50_model.pth"))

/tmp/ipykernel_23/1877423666.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("best_resnet50_model.pth"))


<All keys matched successfully>

In [15]:
# Testing phase
model.eval()
all_labels = []
all_predictions = []

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        all_labels.extend(labels.cpu().numpy())
        all_predictions.extend(predicted.cpu().numpy())

In [16]:
# Calculate metrics
accuracy = accuracy_score(all_labels, all_predictions)
precision = precision_score(all_labels, all_predictions, average='weighted')
recall = recall_score(all_labels, all_predictions, average='weighted')
f1 = f1_score(all_labels, all_predictions, average='weighted')

print(f"Test Accuracy: {accuracy * 100:.2f}%")
print(f"Test Precision: {precision:.4f}")
print(f"Test Recall: {recall:.4f}")
print(f"Test F1 Score: {f1:.4f}")

Test Accuracy: 65.98%
Test Precision: 0.6570
Test Recall: 0.6598
Test F1 Score: 0.6576
